In [1]:
import argparse 
import numpy as np
from PIL import Image, ImageDraw
import math

In [2]:
def read_raw_data(path):
    "Open raw or mipi-raw format file."
    try:
        raw_data = np.fromfile(path, dtype=np.uint8).astype(np.uint16)
        
    except Exception as e:
        print("File read failed Exception:")
    else:
        return raw_data

In [7]:
def mipiraw10toraw(mipiraw10_array: np.ndarray):
    "This is the lossless conversion method of mipi raw to raw."
    mipiraw10_array = mipiraw10_array.reshape((-1,5))
    mipiraw10_array[:,0] = np.left_shift(mipiraw10_array[:,0],2)
    mipiraw10_array[:,1] = np.left_shift(mipiraw10_array[:,1],2)
    mipiraw10_array[:,2] = np.left_shift(mipiraw10_array[:,2],2)
    mipiraw10_array[:,3] = np.left_shift(mipiraw10_array[:,3],2)
    mipiraw10_array[:,0] = mipiraw10_array[:,0] | np.bitwise_and(np.right_shift(mipiraw10_array[:,4],0),0x3)
    mipiraw10_array[:,1] = mipiraw10_array[:,1] | np.bitwise_and(np.right_shift(mipiraw10_array[:,4],2),0x3)
    mipiraw10_array[:,2] = mipiraw10_array[:,2] | np.bitwise_and(np.right_shift(mipiraw10_array[:,4],4),0x3)
    mipiraw10_array[:,3] = mipiraw10_array[:,3] | np.bitwise_and(np.right_shift(mipiraw10_array[:,4],6),0x3)
    return np.delete(mipiraw10_array,-1,axis=1).reshape((1,-1))

In [4]:
rrd = read_raw_data("/home/jc/Project/CViewer/GitLab/CViewer/sample/Sample.raw")

In [5]:
rrd

array([25, 20, 23, ..., 17, 19, 46], dtype=uint16)

In [6]:
rrd.shape

(15000000,)

In [8]:
raw = mipiraw10toraw(rrd)

In [9]:
raw

array([[102,  80,  94, ...,  75,  70,  76]], dtype=uint16)

In [10]:
raw * 2.0

array([[204., 160., 188., ..., 150., 140., 152.]])

In [11]:
raw / 2.0

array([[51. , 40. , 47. , ..., 37.5, 35. , 38. ]])

In [13]:
raw * 0.5

array([[51. , 40. , 47. , ..., 37.5, 35. , 38. ]])

In [14]:
raw

array([[102,  80,  94, ...,  75,  70,  76]], dtype=uint16)

In [21]:
def raw2rgb(raw_array:np.ndarray, width:int, height:int, pattern:str, gain:float):
    "This is the method to convert from raw format to RGB format."
    if abs(1.000000 - gain) > 0.000001:
        raw_array = raw_array*gain       

    channels = {'R': [1, 0, 0], 'G': [0, 1, 0], 'B': [0, 0, 1]}
    channel_marker = np.zeros([height, width, 3])

    for channel, (y, x) in zip(pattern, [(0, 0), (0, 1), (1, 0), (1, 1)]):
        channel_marker[y::2, x::2] = channels[channel]
    channel_marker = channel_marker.reshape([width * height, 3]).T
    rgb_array = (raw_array * channel_marker).T
    rgb_array = rgb_array.reshape([height, width, 3])
    return rgb_array

In [22]:
raw

array([[102,  80,  94, ...,  75,  70,  76]], dtype=uint16)

In [23]:
raw2 = raw2rgb(raw, 4000, 3000, "RGGB", 1.0)

In [24]:
raw2

array([[[102.,   0.,   0.],
        [  0.,  80.,   0.],
        [ 94.,   0.,   0.],
        ...,
        [  0.,  82.,   0.],
        [ 98.,   0.,   0.],
        [  0.,  77.,   0.]],

       [[  0.,  81.,   0.],
        [  0.,   0.,  95.],
        [  0.,  81.,   0.],
        ...,
        [  0.,   0.,  96.],
        [  0.,  79.,   0.],
        [  0.,   0.,  95.]],

       [[ 98.,   0.,   0.],
        [  0.,  82.,   0.],
        [ 92.,   0.,   0.],
        ...,
        [  0.,  84.,   0.],
        [ 96.,   0.,   0.],
        [  0.,  79.,   0.]],

       ...,

       [[  0.,  83.,   0.],
        [  0.,   0.,  92.],
        [  0.,  76.,   0.],
        ...,
        [  0.,   0.,  77.],
        [  0.,  68.,   0.],
        [  0.,   0.,  81.]],

       [[ 95.,   0.,   0.],
        [  0.,  85.,   0.],
        [ 96.,   0.,   0.],
        ...,
        [  0.,  73.,   0.],
        [ 74.,   0.,   0.],
        [  0.,  78.,   0.]],

       [[  0.,  79.,   0.],
        [  0.,   0., 100.],
        [  0.,  

In [25]:
raw2.max()

1023.0

In [26]:
raw2.min()

0.0

In [27]:
rgb_array = (raw2 - raw2.min())/(raw2.max() - raw2.min())*255

In [28]:
rgb_array

array([[[25.42521994,  0.        ,  0.        ],
        [ 0.        , 19.94134897,  0.        ],
        [23.43108504,  0.        ,  0.        ],
        ...,
        [ 0.        , 20.4398827 ,  0.        ],
        [24.42815249,  0.        ,  0.        ],
        [ 0.        , 19.19354839,  0.        ]],

       [[ 0.        , 20.19061584,  0.        ],
        [ 0.        ,  0.        , 23.68035191],
        [ 0.        , 20.19061584,  0.        ],
        ...,
        [ 0.        ,  0.        , 23.92961877],
        [ 0.        , 19.69208211,  0.        ],
        [ 0.        ,  0.        , 23.68035191]],

       [[24.42815249,  0.        ,  0.        ],
        [ 0.        , 20.4398827 ,  0.        ],
        [22.93255132,  0.        ,  0.        ],
        ...,
        [ 0.        , 20.93841642,  0.        ],
        [23.92961877,  0.        ,  0.        ],
        [ 0.        , 19.69208211,  0.        ]],

       ...,

       [[ 0.        , 20.68914956,  0.        ],
        [ 0

In [30]:
rgb_array_8 = rgb_array.astype(np.uint8)

In [31]:
rgb_array_8

array([[[25,  0,  0],
        [ 0, 19,  0],
        [23,  0,  0],
        ...,
        [ 0, 20,  0],
        [24,  0,  0],
        [ 0, 19,  0]],

       [[ 0, 20,  0],
        [ 0,  0, 23],
        [ 0, 20,  0],
        ...,
        [ 0,  0, 23],
        [ 0, 19,  0],
        [ 0,  0, 23]],

       [[24,  0,  0],
        [ 0, 20,  0],
        [22,  0,  0],
        ...,
        [ 0, 20,  0],
        [23,  0,  0],
        [ 0, 19,  0]],

       ...,

       [[ 0, 20,  0],
        [ 0,  0, 22],
        [ 0, 18,  0],
        ...,
        [ 0,  0, 19],
        [ 0, 16,  0],
        [ 0,  0, 20]],

       [[23,  0,  0],
        [ 0, 21,  0],
        [23,  0,  0],
        ...,
        [ 0, 18,  0],
        [18,  0,  0],
        [ 0, 19,  0]],

       [[ 0, 19,  0],
        [ 0,  0, 24],
        [ 0, 20,  0],
        ...,
        [ 0,  0, 18],
        [ 0, 17,  0],
        [ 0,  0, 18]]], dtype=uint8)

In [32]:
Image.fromarray(rgb_array_8).show()

In [34]:
rgb_array_8.max()

255

In [ ]:
rgb_array_8.max()